<a href="https://colab.research.google.com/github/matiaslarregle/visualizaciones-fut/blob/main/Modelo_de_Scouting_Personalizable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [144]:
!pip install pandas LanusStats matplotlib mplsoccer beautifulsoup4 Pillow ScraperFC Spicy math
!pip install LanusStats
!pip install plottable
!pip install matplotlib

  Using cached ScraperFC-3.2.1-py3-none-any.whl.metadata (44 kB)
  Using cached spicy-0.16.0-py2.py3-none-any.whl.metadata (310 bytes)
ERROR: Could not find a version that satisfies the requirement math (from versions: none)
ERROR: No matching distribution found for math


In [145]:
import pandas as pd
import LanusStats as ls
import matplotlib.pyplot as plt
import scipy.stats as stats


from scipy import stats
import math
import numpy as np


from plottable import ColumnDefinition, Table
from plottable.cmap import normed_cmap
from plottable.plots import image

import os

In [146]:
# Verifica los archivos en el entorno de trabajo de Colab
archivos = os.listdir('/content')
print("Archivos en '/content':", archivos)

Archivos en '/content': ['.config', 'thumbnail.png', 'table_Volante Central.png', '.ipynb_checkpoints', 'Argentina Reserve League 2024.csv', 'table_Laterales.png', 'table_Defensa Central.png', 'table_Extremo.png', 'Argentina LPF 2024.csv', 'sample_data']


In [147]:
# Inicializa el DataFrame combinado
df_total = pd.DataFrame()

# Ruta de los archivos cargados
archivos = os.listdir('/content')

# Procesa cada archivo CSV cargado
for archivo in archivos:
    if archivo.endswith('.csv'):
        print(f"Cargando archivo: {archivo}")
        ruta_csv = os.path.join('/content', archivo)

        try:
            # Carga el CSV
            df = pd.read_csv(ruta_csv)

            # Añade la columna 'competicion' con el nombre del archivo sin la extensión
            competicion = archivo.replace('.csv', '')
            df['competicion'] = competicion

            # Combina con el DataFrame total
            df_total = pd.concat([df_total, df], ignore_index=True)
        except Exception as e:
            print(f"Error al cargar {archivo}: {e}")

# Verifica el resultado final
if df_total.empty:
    print("El DataFrame combinado está vacío. Verifica los archivos CSV.")
else:
    # Exploración inicial
    print(df_total.head())
    print(df_total['competicion'].value_counts())

Cargando archivo: Argentina Reserve League 2024.csv
Cargando archivo: Argentina LPF 2024.csv
          Player               Full name  Wyscout id                    Team  \
0     F. Álvarez         Felipe  Álvarez     -411226        River Plate Res.   
1  A. Rimbelitti  Aldo Jesús  Rimbelitti     -520889        River Plate Res.   
2     M. Palacio          Matías Palacio     -478343  Newell's Old Boys Res.   
3     E. Aguirre            Enzo Aguirre     -531476        River Plate Res.   
4     L. Morales        Lisandro Morales     -407171     Unión Santa Fe Res.   

  Team within selected timeframe  \
0               River Plate Res.   
1               River Plate Res.   
2         Newell's Old Boys Res.   
3               River Plate Res.   
4            Unión Santa Fe Res.   

                                           Team logo        Competition  \
0  https://cdn5.wyscout.com/photos/team/public/g2...  Torneo Proyección   
1  https://cdn5.wyscout.com/photos/team/public/g2...  Torne

In [148]:
# Diccionario de mapeo
position_mapping = {
    'CF': 'Delanteros',
    'RB': 'Laterales',
    'RW': 'Extremos',
    'RWF': 'Extremos',
    'LW': 'Extremos',
    'LWF': 'Extremos',
    'LB': 'Laterales',
    'LCB': 'Centrales',
    'RCB': 'Centrales',
    'CB': 'Centrales',
    'LCMF': 'Interiores/mixtos',
    'RCMF': 'Interiores/mixtos',
    'RCMF3': 'Interiores/mixtos',
    'LCMF3': 'Interiores/mixtos',
    'AMF': 'Mediocampistas ofensivos',
    'LAMF': 'Mediocampistas ofensivos',
    'RAMF': 'Mediocampistas ofensivos',
    'DMF': 'Mediocampistas defensivos',
    'LDMF': 'Mediocampistas defensivos',
    'RDMF': 'Mediocampistas defensivos',
    'RWB': 'Laterales',
    'LWB': 'Laterales',
    'GK': 'Arqueros',
    'RB5': 'Laterales',
    'LB5': 'Laterales',
    'LCB3': 'Centrales',
    'RCB3': 'Centrales'
}

# Reemplazar los valores en las columnas correspondientes
df_total['Primary position'] = df_total['Primary position'].replace(position_mapping)
df_total['Secondary position'] = df_total['Secondary position'].replace(position_mapping)
df_total['Third position'] = df_total['Third position'].replace(position_mapping)

# Calcular las frecuencias de cada columna por separado
primary_position_counts = df_total['Primary position'].value_counts()
secondary_position_counts = df_total['Secondary position'].value_counts()
third_position_counts = df_total['Third position'].value_counts()

In [149]:
# Asegurarse de que los valores sean listas (si no lo son) y manejar listas vacías
df_total['Passport country'] = df_total['Passport country'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Extraer el primer país de la lista, manejando listas vacías
df_total['Nacionalidad'] = df_total['Passport country'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

# Verifica los cambios
print(df_total['Nacionalidad'].value_counts())

Nacionalidad
Argentina        1776
Uruguay            61
Paraguay           33
Colombia           30
Chile              14
Peru                5
Brazil              4
United States       4
Mexico              3
Ecuador             3
Venezuela           3
Japan               2
Italy               2
Spain               1
Name: count, dtype: int64


In [150]:
# Define los nombres que deseas mantener
nombres_a_mantener = ['Argentina', 'Uruguay', 'Paraguay', 'Colombia', 'Chile',  'Peru', 'Ecuador', 'Venezuela']

# Define las ligas para las cuales no deseas aplicar el filtro
ligas_sin_filtro = ['Argentina Reserve League 2024', 'Argentina LPF 2024']  # Cambia estas ligas según lo que necesites

# Filtra el DataFrame según la condición de la liga
df_total['Filtrar por nacionalidad'] = df_total['competicion'].apply(lambda liga: liga not in ligas_sin_filtro)

# Filtra el DataFrame para conservar solo las filas de las ligas que no están en ligas_sin_filtro
df_total = df_total[df_total['Filtrar por nacionalidad'] | df_total['Nacionalidad'].isin(nombres_a_mantener)]

# Verifica los primeros 20 valores únicos de la columna filtrada
print(df_total['Nacionalidad'].value_counts().head(20))

Nacionalidad
Argentina    1776
Uruguay        61
Paraguay       33
Colombia       30
Chile          14
Peru            5
Ecuador         3
Venezuela       3
Name: count, dtype: int64


In [151]:
# Reemplaza los NaN en 'Market value' con 'Indeterminado'
df_total['Market value'] = df_total['Market value'].fillna('Indeterminado')

# Filtra los jugadores con un valor de mercado mayor a 2,000,000 o que tengan 'Indeterminado'
df_total = df_total[(df_total['Market value'] >= 0) | (df_total['Market value'] == 'Indeterminado')].reset_index(drop=True)

In [152]:
# Convertir la columna 'Birthday' a formato datetime si no lo está
df_total['Birthday'] = pd.to_datetime(df_total['Birthday'], errors='coerce')

# Calcular la edad restando la fecha de nacimiento de la fecha actual
df_total['Age'] = (pd.to_datetime('today') - df_total['Birthday']).dt.days // 365

# Filtrar los jugadores nacidos en 2006 o después (edad <= 18)
df_total = df_total[df_total['Age'] <= 25]

In [153]:
df_total = df_total[(df_total['Minutes played'] > 270)] .reset_index(drop=True)

In [154]:
estadisticas = [
    "Goals",
    "xG",
    "Assists",
    "xA",
    "Duels per 90",
    "Duels won, %",
    "Successful defensive actions per 90",
    "Defensive duels per 90",
    "Defensive duels won, %",
    "Aerial duels per 90",
    "Aerial duels won, %",
    "Sliding tackles per 90",
    "PAdj Sliding tackles",
    "Shots blocked per 90",
    "Interceptions per 90",
    "PAdj Interceptions",
    "Successful attacking actions per 90",
    "Goals per 90",
    "Non-penalty goals",
    "Non-penalty goals per 90",
    "xG per 90",
    "Head goals",
    "Head goals per 90",
    "Shots",
    "Shots per 90",
    "Goal conversion, %",
    "Assists per 90",
    "Crosses per 90",
    "Accurate crosses, %",
    "Crosses to goalie box per 90",
    "Dribbles per 90",
    "Successful dribbles, %",
    "Offensive duels per 90",
    "Offensive duels won, %",
    "Progressive runs per 90",
    "Accelerations per 90",
    "Fouls suffered per 90",
    "Passes per 90",
    "Accurate passes, %",
    "Forward passes per 90",
    "Accurate forward passes, %",
    "Long passes per 90",
    "Accurate long passes, %",
    "xA per 90",
    "Shot assists per 90",
    "Second assists per 90",
    "Smart passes per 90",
    "Accurate smart passes, %",
    "Key passes per 90",
    "Passes to final third per 90",
    "Accurate passes to final third, %",
    "Passes to penalty area per 90",
    "Accurate passes to penalty area, %",
    "Through passes per 90",
    "Accurate through passes, %",
    "Deep completions per 90",
    "Deep completed crosses per 90",
    "Progressive passes per 90",
    "Accurate progressive passes, %",
    "Clean sheets",
    "Save rate, %",
    "Prevented goals",
    "Prevented goals per 90",
    "Exits per 90",
    "Aerial duels per 90.1",
]

estadisticas_traducidas = [
    "Goles",
    "xG (Goles esperados)",
    "Asistencias",
    "xA (Asistencias esperadas)",
    "Duelos por 90",
    "% de duelos ganados",
    "Acciones defensivas exitosas por 90",
    "Duelos defensivos por 90",
    "% de duelos defensivos ganados",
    "Duelos aéreos por 90",
    "% de duelos aéreos ganados",
    "Barridas por 90",
    "Barridas ajustadas por posesión",
    "Tiros bloqueados por 90",
    "Intercepciones por 90",
    "Intercepciones ajustadas por posesión",
    "Acciones ofensivas exitosas por 90",
    "Goles por 90",
    "Goles sin penales",
    "Goles sin penales por 90",
    "xG por 90",
    "Goles de cabeza",
    "Goles de cabeza por 90",
    "Tiros",
    "Tiros por 90",
    "% de conversión de goles",
    "Asistencias por 90",
    "Centros por 90",
    "% de centros precisos",
    "Centros al área chica por 90",
    "Regates por 90",
    "% de regates exitosos",
    "Duelos ofensivos por 90",
    "% de duelos ofensivos ganados",
    "Carreras progresivas por 90",
    "Aceleraciones por 90",
    "Faltas recibidas por 90",
    "Pases por 90",
    "% de Pases precisos",
    "Pases hacia adelante por 90",
    "% de pases hacia adelante precisos",
    "Pases largos por 90",
    "% de pases largos precisos",
    "xA por 90",
    "Asistencias de tiro por 90",
    "Segundas asistencias por 90",
    "Pases inteligentes por 90",
    "% de pases inteligentes precisos",
    "Pases clave por 90",
    "Pases al tercio final por 90",
    "% de pases precisos al tercio final",
    "Pases al área por 90",
    "% de pases precisos al área",
    "Pases entre líneas por 90",
    "% de pases entre líneas precisos",
    "Pases profundos por 90",
    "Centros profundos completados por 90",
    "Pases progresivos por 90",
    "% de pases progresivos precisos",
    "Vallas invictas",
    "% de atajadas",
    "Goles evitados",
    "Goles evitados por 90",
    "Salidas por 90",
    "Duelos aéreos porr 90",
]

# Crear un diccionario para traducir los nombres
traducciones = dict(zip(estadisticas, estadisticas_traducidas))

# Renombrar las columnas del DataFrame
df_total.rename(columns=traducciones, inplace=True)

In [155]:
estadisticas = [
    "Goles",
    "xG (Goles esperados)",
    "Asistencias",
    "xA (Asistencias esperadas)",
    "Duelos por 90",
    "% de duelos ganados",
    "Acciones defensivas exitosas por 90",
    "Duelos defensivos por 90",
    "% de duelos defensivos ganados",
    "Duelos aéreos por 90",
    "% de duelos aéreos ganados",
    "Barridas por 90",
    "Barridas ajustadas por posesión",
    "Tiros bloqueados por 90",
    "Intercepciones por 90",
    "Intercepciones ajustadas por posesión",
    "Acciones ofensivas exitosas por 90",
    "Goles por 90",
    "Goles sin penales",
    "Goles sin penales por 90",
    "xG por 90",
    "Goles de cabeza",
    "Goles de cabeza por 90",
    "Tiros",
    "Tiros por 90",
    "% de conversión de goles",
    "Asistencias por 90",
    "Centros por 90",
    "Centros al área chica por 90",
    "Regates por 90",
    "% de regates exitosos",
    "Duelos ofensivos por 90",
    "% de duelos ofensivos ganados",
    "Carreras progresivas por 90",
    "Aceleraciones por 90",
    "Faltas recibidas por 90",
    "Pases por 90",
    "% de Pases precisos",
    "Pases hacia adelante por 90",
    "Pases largos por 90",
    "% de pases largos precisos",
    "xA por 90",
    "Asistencias de tiro por 90",
    "Segundas asistencias por 90",
    "Pases inteligentes por 90",
    "Pases clave por 90",
    "Pases al tercio final por 90",
    "Pases al área por 90",
    "Pases entre líneas por 90",
    "Pases profundos por 90",
    "Centros profundos completados por 90",
    "Pases progresivos por 90",
    "Vallas invictas",
    "% de atajadas",
    "Goles evitados",
    "Goles evitados por 90",
    "Salidas por 90",
    "Duelos aéreos porr 90",
]

In [156]:
# Definir las posiciones y las estadísticas relevantes
posiciones = {
    "Defensa Central": {
        "estadisticas": [
            "% de duelos aéreos ganados", "% de duelos defensivos ganados", "Intercepciones por 90",
            "Acciones defensivas exitosas por 90", "Tiros bloqueados por 90", "Barridas por 90"
        ],
        "pesos": [0.3, 0.25, 0.2, 0.1, 0.1, 0.05],
        "primary_position": ["Centrales"]
    },
    "Laterales": {
        "estadisticas": [
            "% de centros precisos", "% de duelos defensivos ganados", "Intercepciones por 90",
            "Pases clave por 90", "Regates por 90", "Centros al área chica por 90"
        ],
        "pesos": [0.3, 0.25, 0.15, 0.1, 0.1, 0.1],
        "primary_position": ["Laterales"]
    },
    "Volante Central": {
        "estadisticas": [
            "Pases clave por 90", "% de duelos ganados", "% de duelos defensivos ganados", "xA por 90",
            "Pases largos por 90"
        ],
        "pesos": [0.3, 0.25, 0.2, 0.15, 0.1],
        "primary_position": ["Mediocampistas defensivos"]
    },
    "Volante Interior": {
        "estadisticas": [
            "Pases clave por 90", "xA por 90", "Intercepciones por 90", "% de duelos defensivos ganados",
            "Regates por 90", "Faltas recibidas por 90", "Carreras progresivas por 90"
        ],
        "pesos": [0.25, 0.2, 0.2, 0.15, 0.1, 0.05, 0.05],
        "primary_position": ["Mediocampistas defensivos", "Mediocampista Mixto", "Mediocampistas ofensivos"]
    },
    "Extremo": {
        "estadisticas": [
            "% de duelos ganados", "% de centros precisos", "xA por 90", "Regates por 90",
            "Pases clave por 90", "Pases al área por 90"
        ],
        "pesos": [0.3, 0.25, 0.2, 0.1, 0.1, 0.05],
        "primary_position": ["Extremos", "Mediocampistas ofensivos"]
    },
    "Delantero Centro": {
        "estadisticas": [
            "% de duelos aéreos ganados", "xG por 90", "% de duelos ofensivos ganados", "Goles por 90",
            "Tiros por 90"
        ],
        "pesos": [0.25, 0.2, 0.2, 0.15, 0.1],
        "primary_position": ["Delanteros"]
    }
}

# Ponderaciones por liga
ponderaciones_liga = {
    "Argentina LPF 2024": 1,
    "Argentina Reserve League 2024": 0.75,
}

# Función para calcular el score de un jugador
def calcular_score(row, posicion):
    # Obtener el factor de ponderación de la liga
    factor_liga = ponderaciones_liga.get(row["competicion"], 1.0)  # Por defecto, 1.0 si la liga no está en el diccionario

    # Calcular el score ponderado
    score = 0
    for stat, peso in zip(posiciones[posicion]["estadisticas"], posiciones[posicion]["pesos"]):
        score += (row[stat] * peso) ** 2  # Elevar al cuadrado el valor ponderado de cada estadística

    return np.sqrt(score) * factor_liga  # Multiplicar por el factor de la liga

# Filtrar jugadores por posición primaria
def filtrar_por_posicion(df, posicion):
    # Filtrar jugadores que tengan la posición primaria correspondiente
    return df[df['Primary position'].isin(posiciones[posicion]["primary_position"])]

# Calcular el score para cada jugador en la posición seleccionada
def calcular_scores(df, posicion):
    df_filtrado = filtrar_por_posicion(df, posicion)
    df_filtrado['Score'] = df_filtrado.apply(lambda row: calcular_score(row, posicion), axis=1)
    df_filtrado = df_filtrado.sort_values(by='Score', ascending=False)
    return df_filtrado

# Lista de posiciones a comparar
posiciones_a_comparar = list(posiciones.keys())

# Iterar sobre cada posición y calcular el score
for posicion in posiciones_a_comparar:
    df_filtrado = calcular_scores(df_total, posicion)
    print(f"\nJugadores ordenados por score para la posición {posicion}:")
    print(df_filtrado[['Full name', 'competicion', 'Score']])


Jugadores ordenados por score para la posición Defensa Central:
                             Full name                    competicion  \
727                 Francisco Petrasso             Argentina LPF 2024   
646               Tiago Tomás Palacios             Argentina LPF 2024   
658                      Gian Nardelli             Argentina LPF 2024   
541           Francisco Fabián Álvarez             Argentina LPF 2024   
490               Santiago Leonel Sosa             Argentina LPF 2024   
..                                 ...                            ...   
90         Matias Oscar Funes Martínez  Argentina Reserve League 2024   
55          Julián Ezequiel Echeverría  Argentina Reserve League 2024   
400                    Facundo Benítez  Argentina Reserve League 2024   
177  Lautaro Ángel Ezequiel Montenegro  Argentina Reserve League 2024   
469             Tomás Julián Fernández  Argentina Reserve League 2024   

         Score  
727  29.529181  
646  29.294201  
658  28

In [157]:
top_5_jugadores = []

# Iterar sobre cada posición y calcular el score
for posicion in posiciones_a_comparar:
    df_filtrado = calcular_scores(df_total, posicion)

    # Obtener los top 5 jugadores para la posición
    top_5 = df_filtrado.head(5)

    # Añadir una columna con el nombre de la posición
    top_5['Posición'] = posicion

    # Agregar los top 5 de esta posición a la lista
    top_5_jugadores.append(top_5)

# Concatenar todos los top 5 de las posiciones
df_top_5 = pd.concat(top_5_jugadores)

df_plot = df_top_5.sort_values(by='Score', ascending=False).head(15)
# Definir las columnas que quieres conservar
columnas_a_conservar = ['Full name', 'Score', 'Posición']

# Filtrar el DataFrame para dejar solo esas columnas
df_plot = df_plot.filter(items=columnas_a_conservar)
df_plot['Score'] = round(df_plot['Score'],2)

In [158]:
import matplotlib.pyplot as plt
from plottable import ColumnDefinition, Table
from plottable.cmap import normed_cmap
import matplotlib

# Configuración de colores y estilo
bg_color = "#FFFFFF"  # Fondo blanco
text_color = "#000000"  # Texto negro

row_colors = {
    "top2": "#E1FABC",
    "top4": "#FFFC97",
    "relegation": "#E79A9A",
    "even": "#E2E2E1",
    "odd": "#B3B0B0",
}

# Configuración global de matplotlib
plt.rcParams["text.color"] = text_color
plt.rcParams["font.family"] = "monospace"

# Filtrar las posiciones únicas
posiciones_unicas = df_plot['Posición'].unique()

# Iterar sobre cada posición única
for posicion in posiciones_unicas:
    # Filtrar el DataFrame por la posición
    df_posicion = df_plot[df_plot['Posición'] == posicion]

    # Verificar si hay datos para esa posición
    if not df_posicion.empty:
        # Definir las columnas para la tabla
        col_defs = [
            ColumnDefinition(
                name="Full name",
                textprops={"ha": "right"},
                width=5
            ),
            ColumnDefinition(
                name="Score",
                textprops={"ha": "center", "color": "#000000", "weight": "bold"},
                cmap=normed_cmap(df_posicion["Score"], cmap=matplotlib.cm.PiYG, num_stds=2)
            )
        ]

        # Crear la figura para la tabla
        fig, ax = plt.subplots(figsize=(7, 13))
        fig.set_facecolor(bg_color)
        ax.set_facecolor(bg_color)

        # Crear la tabla usando plottable
        table = Table(
            df_posicion,
            column_definitions=col_defs,
            index_col="Full name",
            row_dividers=True,
            row_divider_kw={"linewidth": 1, "linestyle": (0, (1, 5))},
            footer_divider=True,
            textprops={"fontsize": 14},
            col_label_divider_kw={"linewidth": 1, "linestyle": "-"},
            column_border_kw={"linewidth": 3, "linestyle": "-"},
            ax=ax,
        ).autoset_fontcolors(colnames=["Full name", "Score"])  # Esto ajusta el color de la fuente basado en el cmap

        # Aplicar los colores de las filas manualmente
        for idx, row in df_posicion.iterrows():
            if idx < 2:
                row_color = row_colors["top2"]
            elif idx < 4:
                row_color = row_colors["top4"]
            elif idx % 2 == 0:
                row_color = row_colors["even"]
            else:
                row_color = row_colors["odd"]

            # Cambiar el color de la fila
            ax.add_patch(plt.Rectangle((0, idx), 1, 1, color=row_color, lw=0))  # Cambiar el color de la fila

        # Título de la tabla
        plt.title(f'Ranking posición: {posicion}', size=18)

        # Guardar la imagen con el nombre de la posición
        plt.savefig(f'table_{posicion}.png', dpi=300, transparent=True)
        plt.close()

In [159]:
# Función para calcular el score de un jugador
def calcular_score(row, posicion):
    score = 0
    for stat, peso in zip(posiciones[posicion]["estadisticas"], posiciones[posicion]["pesos"]):
        score += (row[stat] * peso) ** 2  # Elevar al cuadrado el valor ponderado de cada estadística
    return np.sqrt(score)  # Obtener la raíz cuadrada de la suma de los cuadrados

# Filtrar jugadores por equipo y posición primaria
def filtrar_por_equipo_y_posicion(df, equipo, posicion):
    # Filtrar jugadores que sean del equipo y que tengan la posición primaria correspondiente
    return df[(df['Primary position'].isin(posiciones[posicion]["primary_position"])) &
              (df['Team within selected timeframe'] == equipo)]

# Calcular el score para cada jugador en la posición seleccionada
def calcular_scores(df, equipo, posicion):
    df_filtrado = filtrar_por_equipo_y_posicion(df, equipo, posicion)
    df_filtrado['Score'] = df_filtrado.apply(lambda row: calcular_score(row, posicion), axis=1)
    df_filtrado = df_filtrado.sort_values(by='Score', ascending=False).reset_index(drop=True)  # Resetear índice
    return df_filtrado

# Lista de posiciones a comparar
posiciones_a_comparar = list(posiciones.keys())

# Nombre del equipo
equipo = "Rosario Central"

# Crear un diccionario para almacenar los percentiles de los mejores jugadores
percentiles_mejores_jugadores = {}

# Determinar la liga del equipo
liga_equipo = df_total[df_total['Team within selected timeframe'] == equipo]['competicion'].iloc[0]

# Iterar sobre cada posición y calcular el percentil
for posicion in posiciones_a_comparar:
    df_filtrado = calcular_scores(df_total, equipo, posicion)

    # Si no hay jugadores en esta posición para el equipo, continuar
    if df_filtrado.empty:
        percentiles_mejores_jugadores[posicion] = None
        continue

    # Tomar el mejor jugador de mi equipo en esta posición
    mejor_jugador = df_filtrado.iloc[0]  # El mejor jugador en esa posición
    mejor_jugador_score = mejor_jugador['Score']

    # Encontrar el ranking general de esa posición dentro de la misma liga
    df_general = df_total[
        (df_total['Primary position'].isin(posiciones[posicion]["primary_position"])) &
        (df_total['competicion'] == liga_equipo)
    ]
    df_general['Score'] = df_general.apply(lambda row: calcular_score(row, posicion), axis=1)
    df_general = df_general.sort_values(by='Score', ascending=False).reset_index(drop=True)  # Resetear índice

    # Calcular el percentil del mejor jugador de mi equipo
    rank = df_general[df_general['Score'] > mejor_jugador_score].shape[0] + 1
    percentil = (len(df_general) - rank + 1) / len(df_general) * 100

    # Guardar el percentil
    percentiles_mejores_jugadores[posicion] = percentil

# Determinar la posición donde el percentil del mejor jugador es más bajo
peor_posicion = min(
    {k: v for k, v in percentiles_mejores_jugadores.items() if v is not None},
    key=percentiles_mejores_jugadores.get
)

# Mostrar los resultados
print("Percentiles de los mejores jugadores de mi equipo por posición (comparando con jugadores de la misma liga):")
for posicion, percentil in percentiles_mejores_jugadores.items():
    if percentil is not None:
        print(f"En la posición {posicion}, el mejor jugador está en el percentil: {percentil:.2f}%")
    else:
        print(f"En la posición {posicion}, no hay jugadores de mi equipo.")

print(f"\nLa posición donde el percentil del mejor jugador es más bajo es: {peor_posicion} (percentil {percentiles_mejores_jugadores[peor_posicion]:.2f}%) POTENCIAL POSICIÓN A REFORZAR")


Percentiles de los mejores jugadores de mi equipo por posición (comparando con jugadores de la misma liga):
En la posición Defensa Central, el mejor jugador está en el percentil: 54.00%
En la posición Laterales, el mejor jugador está en el percentil: 36.07%
En la posición Volante Central, el mejor jugador está en el percentil: 89.29%
En la posición Volante Interior, el mejor jugador está en el percentil: 85.14%
En la posición Extremo, el mejor jugador está en el percentil: 74.44%
En la posición Delantero Centro, no hay jugadores de mi equipo.

La posición donde el percentil del mejor jugador es más bajo es: Laterales (percentil 36.07%) POTENCIAL POSICIÓN A REFORZAR
